In [ ]:
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import numpy as np
import imutils
import os
import time

In [ ]:
pdf_photo_model = tf.saved_model.load('pdf_vs_photo_v1/exported_pdf_vs_photo')
print(pdf_photo_model.signatures['serving_default'])  

In [ ]:
sar_model = tf.saved_model.load('small_rotation_v4/epoch-5-small-rotation/exported_small_rotation')
print(sar_model.signatures['serving_default'])

In [ ]:
# CHANGE THESE PATHS TO YOUR OWN
###############################################################
input_path = '/home/asus/stuDYING/IT/Thesis/from_server/heic_fol/030124'
output_path = '/home/asus/stuDYING/IT/Thesis/preprocessing_pipeline/out_file/heic_fol/030124'
##############################################################

In [ ]:
if not os.path.exists('unwarping_output'):
    os.makedirs('unwarping_output')

unwarping_output = os.path.join(os.getcwd(), 'unwarping_output')
print(unwarping_output)

In [ ]:
def rotate_small_angle(img, angle):
    rotated = imutils.rotate_bound(img, int(angle))
    return rotated

def resizeAndPad(img, size, padColor=0):

    h, w = img.shape[:2]
    sh, sw = size

    if h > sh or w > sw:
        interp = cv2.INTER_AREA
    else:
        interp = cv2.INTER_CUBIC

    aspect = w/h

    if aspect > 1:
        new_w = sw
        new_h = np.round(new_w/aspect).astype(int)
        pad_vert = (sh-new_h)/2
        pad_top, pad_bot = np.floor(pad_vert).astype(int), np.ceil(pad_vert).astype(int)
        pad_left, pad_right = 0, 0
    elif aspect < 1:
        new_h = sh
        new_w = np.round(new_h*aspect).astype(int)
        pad_horz = (sw-new_w)/2
        pad_left, pad_right = np.floor(pad_horz).astype(int), np.ceil(pad_horz).astype(int)
        pad_top, pad_bot = 0, 0
    else: # square image
        new_h, new_w = sh, sw
        pad_left, pad_right, pad_top, pad_bot = 0, 0, 0, 0

    if len(img.shape) == 3 and not isinstance(padColor, (list, tuple, np.ndarray)):
        padColor = [padColor]*3

    scaled_img = cv2.resize(img, (new_w, new_h), interpolation=interp)
    scaled_img = cv2.copyMakeBorder(scaled_img, pad_top, pad_bot, pad_left, pad_right, borderType=cv2.BORDER_CONSTANT, value=padColor)

    return scaled_img

In [ ]:
total_time = 0
num_images = 0

for image in os.listdir(input_path):
    try:
        if image == '.gitkeep':
            continue

        start_time = time.time()

        img_path = os.path.join(input_path, image)

        print('Processing image: ' + img_path)

        img = cv2.imread(img_path)

        classes = ['pdf', 'photo']

        # plt.imshow(img)
        # plt.show()
        h, w, _ = img.shape
        img = cv2.resize(img, (480, 480))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        input_tensor = tf.convert_to_tensor(np.array([img]), dtype=tf.float32)

        result = pdf_photo_model.signatures['serving_default'](input_tensor)['dense_1'][0]

        print(result)
        print('Classified as class: ' + classes[np.argmax(result)])

        end_time = time.time()

        pdf_photo_time = end_time - start_time

        print('Classification time: ', pdf_photo_time)


        if classes[np.argmax(result)] == 'pdf':
            cv2.imwrite(os.path.join(output_path, image), img)
            print('-' * 50)
            continue
        else:

            start_time = time.time()

            args = f'./test -idir={img_path} -odir={unwarping_output}'
            os.system(args)

            end_time = time.time()

            unwarping_time = end_time - start_time

            print('Unwarping time: ', unwarping_time)

            # print(unwarping_output + '/' + image.split(".")[0] + '_remap.png')

            unwarped_img = cv2.imread(unwarping_output + '/' + image.split(".")[0] + '_remap.png')

            start_time = time.time()

            rs_img = resizeAndPad(unwarped_img, (480, 480))
            rs_img = cv2.cvtColor(rs_img, cv2.COLOR_BGR2RGB)
            rs_img = rs_img / 255.0
            print(rs_img.shape)
            input_tensor = tf.convert_to_tensor(np.array([rs_img]), dtype=tf.float32)

            result = sar_model.signatures['serving_default'](input_tensor)['output_0'][0]

            rotated = rotate_small_angle(img, result[0])

            # print('Rotated by: ', result[0], ' degrees')

            rotate_time = time.time() - start_time

            print('Rotatingtime: ', rotate_time)

            cv2.imwrite(os.path.join(output_path, image), rotated)
                
            print('-' * 50)
    except Exception:
        pass